<a href="https://colab.research.google.com/github/PDNow-Research/PDNow/blob/main/HandPD/SVM_Exact_Replication_Experiment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Data Science
import re
import csv
import json
import itertools
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# General
import os
import sys
import time
import math
import random
from datetime import date
import warnings
current_date = date.today()
warnings.filterwarnings("ignore")

# SVM
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, average_precision_score, classification_report

In [3]:
#NewSpiral is from the new HandPD dataset. The new one also comes with images and such, but we are using the data that has the exact same features from old HandPd dataset, except also includes patient ID.
spiral_df = pd.read_csv('/content/drive/My Drive/Data/HandPD-Replication/NewSpiral.csv')

In [4]:
spiral_df.shape

(264, 16)

In [5]:
spiral_df['CLASS_TYPE'].value_counts()

1    140
2    124
Name: CLASS_TYPE, dtype: int64

In [6]:
meander_df = pd.read_csv('/content/drive/My Drive/Data/HandPD-Replication/NewMeander.csv')

In [7]:
meander_df['ID_PATIENT'].value_counts()

5      8
31     8
23     8
305    4
17     4
      ..
143    4
138    4
127    4
104    4
1      4
Name: ID_PATIENT, Length: 63, dtype: int64

In [8]:
meander_df['ID_PATIENT'][meander_df['_ID_EXAM'] == "P26"] = 32

In [9]:
meander_df.columns

Index(['_ID_EXAM', 'IMAGE_NAME', 'ID_PATIENT', 'CLASS_TYPE', 'GENDER',
       'RIGH/LEFT-HANDED', 'AGE', 'RMS', 'MAX_BETWEEN_ET_HT',
       'MIN_BETWEEN_ET_HT', 'STD_DEVIATION_ET_HT', 'MRT', 'MAX_HT', 'MIN_HT',
       'STD_HT', 'CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT'],
      dtype='object')

In [10]:
spiral_df['ID_PATIENT'].unique().tolist()

[59,
 76,
 301,
 102,
 305,
 104,
 127,
 297,
 299,
 98,
 78,
 80,
 86,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 143,
 31,
 38,
 47,
 53,
 235,
 253,
 261,
 268,
 273,
 281,
 247,
 149,
 157,
 165,
 186,
 192,
 196,
 207,
 218,
 203,
 224,
 153,
 176,
 181,
 230,
 138,
 187]

In [11]:
spiral_df[spiral_df['ID_PATIENT'] == 31]

,_ID_EXAM,IMAGE_NAME,ID_PATIENT,CLASS_TYPE,GENDER,RIGH/LEFT-HANDED,AGE,RMS,MAX_BETWEEN_ET_HT,MIN_BETWEEN_ET_HT,STD_DEVIATION_ET_HT,MRT,MAX_HT,MIN_HT,STD_HT,CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT
141,P2,mea1-P2.jpg,31,2,F,R,59,3726.488067,5893.347008,37785.94529,0.306602,24.403343,179.554888,0.008474,1739.851476,0.267742
164,P26,mea1-P26.jpg,31,2,F,L,59,4587.166408,6561.231877,37033.96196,0.018378,22.328549,184.806500,0.057158,1388.315960,0.266871
172,P2,mea2-P2.jpg,31,2,F,R,59,3469.841548,5282.395889,33582.16916,0.000129,18.103416,170.531514,0.045752,1130.591703,0.204545
195,P26,mea2-P26.jpg,31,2,F,L,59,6210.596735,7365.102809,37813.01828,0.002359,20.958406,160.395258,0.024497,1281.094994,0.128205
203,P2,mea3-P2.jpg,31,2,F,R,59,5066.989293,7463.980609,35307.01259,0.000000,22.643151,176.392744,0.008650,1614.293957,0.246711
226,P26,mea3-P26.jpg,31,2,F,L,59,6544.571924,8612.729092,36154.08597,0.013198,27.237956,180.306815,0.003577,1761.998890,0.243421
234,P2,mea4-P2.jpg,31,2,F,R,59,3748.920825,6607.203671,36244.36135,0.001141,24.606020,179.208863,0.028543,1782.895068,0.263158
257,P26,mea4-P26.jpg,31,2,F,L,59,4921.267695,6948.380564,35304.08954,0.017636,25.699685,182.482156,0.055527,1698.122391,0.235915


From the above, we can see that there are two patients with the ID of 31. So we're going to give one of them an ID of 32 instead, as that ID does not belong to any other.

In [12]:
spiral_df['ID_PATIENT'][spiral_df['_ID_EXAM'] == "P26"] = 32

In [13]:
spiral_df[spiral_df['ID_PATIENT'] == 32]

,_ID_EXAM,IMAGE_NAME,ID_PATIENT,CLASS_TYPE,GENDER,RIGH/LEFT-HANDED,AGE,RMS,MAX_BETWEEN_ET_HT,MIN_BETWEEN_ET_HT,STD_DEVIATION_ET_HT,MRT,MAX_HT,MIN_HT,STD_HT,CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT
164,P26,mea1-P26.jpg,32,2,F,L,59,4587.166408,6561.231877,37033.96196,0.018378,22.328549,184.806500,0.057158,1388.315960,0.266871
195,P26,mea2-P26.jpg,32,2,F,L,59,6210.596735,7365.102809,37813.01828,0.002359,20.958406,160.395258,0.024497,1281.094994,0.128205
226,P26,mea3-P26.jpg,32,2,F,L,59,6544.571924,8612.729092,36154.08597,0.013198,27.237956,180.306815,0.003577,1761.998890,0.243421
257,P26,mea4-P26.jpg,32,2,F,L,59,4921.267695,6948.380564,35304.08954,0.017636,25.699685,182.482156,0.055527,1698.122391,0.235915


##**Preprocessing**

###Normalization

fi' = (fi - avg)/std

In [84]:
# normalization with the paper's method (formula above)
def feature_normalization(df):
  df_dup = df.copy()

  df1 = df_dup[['RMS', 'MAX_BETWEEN_ET_HT',
       'MIN_BETWEEN_ET_HT', 'STD_DEVIATION_ET_HT', 'MRT', 'MAX_HT', 'MIN_HT',
       'STD_HT', 'CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT']]

  df2 = df_dup[['_ID_EXAM', 'IMAGE_NAME', 'ID_PATIENT', 'CLASS_TYPE', 'GENDER',
       'RIGH/LEFT-HANDED', 'AGE']]
  avg_dev = df1.mad(axis = 0)
  std_dev = df1.std(axis = 0)

  df1 = df1.sub(avg_dev)
  df1 = df1.divide(std_dev)

  return df2, df1

In [85]:
# normalizing spiral_df
spiral_df = pd.concat(feature_normalization(spiral_df), axis = 1)

In [86]:
# normalizing X_meander
meander_df =  pd.concat(feature_normalization(meander_df), axis = 1)

###Data Splitting
"The first one [experiment] uses 75% of the dataset for training purposes and the remaining 25% for the classification phase. However, instead of partitioning the dataset randomly, we created four subsets in order to guarantee that each individual will be represented in the dataset with its 3 spirals/meanders, with the remaining one being used for classification purposes. In this experiment, the spiral- and meander-based datasets are used individually."

In [87]:
def split_data(df, dfarr):
  df_IDs = df['ID_PATIENT'].unique().tolist()

  for x in df_IDs:
    for i in range(len(dfarr)):
      row = df[df['ID_PATIENT'] == x].iloc[i]
      dfarr[i] = dfarr[i].append(row)

  return dfarr

In [88]:
def represented_train_test_split(df, arrnum): # df is array that gets split and arrnum is which fold is used in test (other 3 used in train)

  test = pd.DataFrame(columns = df.columns)
  train = pd.DataFrame(columns = df.columns)

  dfarr = [] # creating list of lists with 4 lists
  for i in range(4):
    dfarr.append(pd.DataFrame(columns = df.columns))

  # Some patients may have multiple test IDs so we must base this on Patient ID, which can be same for PD and Ctrl groups. So we split them.
  ctrl_df = df.query('CLASS_TYPE == 1')
  pd_df = df.query('CLASS_TYPE == 2')

  # we can append very easily because each of the IDs are different in ctrl_df and pd_df, since we split from X_spiral, which has all unique IDs
  dfarr = split_data(ctrl_df, dfarr)
  dfarr = split_data(pd_df, dfarr)


  test = dfarr[arrnum]
  test = test.reset_index(drop=True)

  dfarr.pop(arrnum)

  train = pd.concat(dfarr) # after droppage of arrnum used for test
  train = train.reset_index(drop=True)

  return test, train

In [89]:
meander_test, meander_train = represented_train_test_split(meander_df, 0)

In [90]:
meander_test['CLASS_TYPE'].value_counts()

1    35
2    31
Name: CLASS_TYPE, dtype: int64

In [91]:
meander_train.shape

(198, 16)

In [92]:
meander_test['ID_PATIENT'].value_counts()

5      2
23     2
14     1
17     1
38     1
      ..
53     1
181    1
305    1
176    1
1      1
Name: ID_PATIENT, Length: 64, dtype: int64

In [93]:
meander_train['ID_PATIENT'].value_counts()

5      6
23     6
14     3
17     3
38     3
      ..
53     3
181    3
305    3
176    3
1      3
Name: ID_PATIENT, Length: 64, dtype: int64

As can be seen, we end up with only 2 IDs that are duplicated in PD and control patients: 5 and 23.

###Train-Test-Split into X and y

In [94]:
meander_train.columns

Index(['_ID_EXAM', 'IMAGE_NAME', 'ID_PATIENT', 'CLASS_TYPE', 'GENDER',
       'RIGH/LEFT-HANDED', 'AGE', 'RMS', 'MAX_BETWEEN_ET_HT',
       'MIN_BETWEEN_ET_HT', 'STD_DEVIATION_ET_HT', 'MRT', 'MAX_HT', 'MIN_HT',
       'STD_HT', 'CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT'],
      dtype='object')

In [95]:
def final_train_test_split(train, test):
  # creating our final training dataset with spiral drawings
  X_train = train[['RMS', 'MAX_BETWEEN_ET_HT', 'MIN_BETWEEN_ET_HT', 'STD_DEVIATION_ET_HT', 'MRT', 'MAX_HT', 'MIN_HT', 'STD_HT', 'CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT']]
  y_train = train['CLASS_TYPE']

  y_train = pd.DataFrame(y_train)


  # creating our final testing dataset with spiral drawings
  X_test = test[['RMS', 'MAX_BETWEEN_ET_HT', 'MIN_BETWEEN_ET_HT', 'STD_DEVIATION_ET_HT', 'MRT', 'MAX_HT', 'MIN_HT', 'STD_HT', 'CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT']]
  y_test = test['CLASS_TYPE']

  y_test = pd.DataFrame(y_test)

  return X_train, y_train, X_test, y_test

In [96]:
X_meander_train, y_meander_train, X_meander_test, y_meander_test = final_train_test_split(meander_train, meander_test)

In [97]:
# converting to type SVM can understand - int instead of object

y_meander_train['CLASS_TYPE'] = pd.to_numeric(y_meander_train['CLASS_TYPE'])
y_meander_test['CLASS_TYPE'] = pd.to_numeric(y_meander_test['CLASS_TYPE'])

###Quick Check

In [98]:
y_meander_train.value_counts()

CLASS_TYPE
1             105
2              93
dtype: int64

In [99]:
y_meander_test.value_counts()

CLASS_TYPE
1             35
2             31
dtype: int64

##**SVM Implementation**

###**Training with Cross-Val**

In [100]:
def cross_validation_test(df):

  acc_list = [] * 4
  X_test_list = [] * 4
  y_testset_list = [] * 4

  # no need for normalization. I've already normalized meander_df, which I will be passing into here. feature_normalization(df)
  # changing the data I feed in

  for i in range(4):

    # the train_test_splitting
    test, train = represented_train_test_split(df, i)
    X_train, y_train, X_test, y_test = final_train_test_split(train, test)

    y_testset_list.append(y_test)

    # converting labels from objects to numeric
    y_train['CLASS_TYPE'] = pd.to_numeric(y_train['CLASS_TYPE'])
    y_test['CLASS_TYPE'] = pd.to_numeric(y_test['CLASS_TYPE'])

    # the model running
    clf = SVC(kernel = 'rbf', probability = True, class_weight='balanced')
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    y_pred = pd.Series(y_pred)

    y_proba = clf.predict_proba(X_test)

    # y_proba becomes the probability of being in Class 1 (control)
    y_proba = [x for list in y_proba for x in list]
    y_proba = y_proba[0::2] 

    acc_list.append(clf.score(X_test, y_test))

    X_test["Predictions"] = y_pred
    X_test["Certainity"] = y_proba 

    X_test_list.append(X_test)

  return y_testset_list, X_test_list, acc_list

In [101]:
y_list, X_test_list, acc_list = cross_validation_test(meander_df)

In [ ]:
X_test_list

In [103]:
acc_list

[0.7727272727272727, 0.7878787878787878, 0.7424242424242424, 0.696969696969697]

In [104]:
acc_list = np.asarray(acc_list)

In [105]:
acc_list.mean()

0.75

In [106]:
acc_list.std()

0.03471648253754421

In [107]:
for i in range(4):
  target_names = ['Control', 'PD']
  results = classification_report(y_list[i], X_test_list[i]["Predictions"], target_names = target_names, output_dict=True)
  results = pd.DataFrame(results).transpose()
  conf_mat = confusion_matrix(y_list[i], X_test_list[i]["Predictions"])

  print (conf_mat)
  print('  ')

[[23 12]
 [ 3 28]]
  
[[22 13]
 [ 1 30]]
  
[[18 17]
 [ 0 31]]
  
[[15 20]
 [ 0 31]]
  


In [108]:
control_acc = np.asarray([23/35, 22/35, 18/35, 15/35])
pd_acc = np.asarray([28/31, 30/31, 31/31, 31/31])

In [111]:
control_acc.mean()
# pd_acc.mean()

0.557142857142857

In [112]:
control_acc.std()
# pd_acc.std()

0.09147320339189785

#### Specific Test

In [ ]:
clf = SVC(kernel = 'rbf', probability = True, class_weight = 'balanced')
clf.fit(X_meander_train, y_meander_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

To note, it may be useful to look at the weights for this model (see which features are most valuable), but these are only relevant for a linear kernel. RBF kernel does not have relevant/interpretable weights. And of course, linear kernel is not very helpful and good in predicting (51% accuracy). The features are not linearly separable.

In [ ]:
y_meander_pred = clf.predict(X_meander_test)
y_meander_proba = clf.predict_proba(X_meander_test)

In [ ]:
y_meander_pred = pd.Series(y_meander_pred)

In [ ]:
len(y_meander_pred[y_meander_pred==2])

54

##**Results**

#### Deviance of probabilites from 100/0 depending on class

In [ ]:
# flattening y_meander_proba & removing the second percentage we don't need

y_meander_proba = [x for list in y_meander_proba for x in list]
y_meander_proba = y_meander_proba[0::2] # takes every other element starting from the first element

In [ ]:
testing = y_meander_proba[i for i in y_meander_proba if i < 0.2775]

SyntaxError: ignored

In [ ]:
len(y_meander_proba)

In [ ]:
# calculating standard dev 

proba1 = [i for i in y_meander_proba if i > 0.77]
proba2 = [i for i in y_meander_proba if i < 0.77]

In [ ]:
len(proba2)

In [ ]:
print(proba2)

In [ ]:
meanval =0

for i in proba1:
  meanval += (i)

meanval /= len(proba1)

meanval

In [ ]:
len(proba2)

#### Y Meander Results

In [ ]:
# y_meander_test = y_meander_test['CLASS_TYPE'].astype(str).astype(int)

In [ ]:
y_meander_test.dtypes

CLASS_TYPE    int64
dtype: object

In [ ]:
clf.score(X_meander_test, y_meander_test)

0.6212121212121212

In [ ]:
target_names = ['Control', 'PD']
results = classification_report(y_meander_test, y_meander_pred, target_names = target_names, output_dict=True)
results = pd.DataFrame(results).transpose()
conf_mat = confusion_matrix(y_meander_test, y_meander_pred)

In [ ]:
results

,precision,recall,f1-score,support
Control,0.916667,0.314286,0.468085,35.000000
PD,0.555556,0.967742,0.705882,31.000000
accuracy,0.621212,0.621212,0.621212,0.621212
macro avg,0.736111,0.641014,0.586984,66.000000
weighted avg,0.747054,0.621212,0.579778,66.000000


In [ ]:
conf_mat

array([[11, 24],
       [ 1, 30]])

In [ ]:
TN, FP, FN, TP = conf_mat.ravel()

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)

# Specificity or true negative rate
TNR = TN/(TN+FP) 

# Precision or positive predictive value
PPV = TP/(TP+FP)

# Negative predictive value
NPV = TN/(TN+FN)

# Fall out or false positive rate
FPR = FP/(FP+TN)

# False negative rate
FNR = FN/(TP+FN)

# False discovery rate
FDR = FP/(TP+FP)

print("TP: ", TP)
print("TN: ", TN)
print("FP: ", FP)
print("FN: ", FN)

print("Sensitivity: ", TPR)
print("Specificity: ", TNR)
print("NPV: ", NPV)
print("PPV: ", PPV)

TP:  30
TN:  11
FP:  24
FN:  1
Sensitivity:  0.967741935483871
Specificity:  0.3142857142857143
NPV:  0.9166666666666666
PPV:  0.5555555555555556


Can we somehow get the weights for the model to learn which features are considered most useful?